In [3]:
# Importing Required Libraries
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [14]:
# Loading data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [4]:
# Initialising variables
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) * n_conv2

In [5]:
# Initialising weights and biases for all layers
weights = {
    'wc1' : tf.Variable(tf.random_normal([conv1_k, conv1_k, input_channels, n_conv1])),
    'wc2' : tf.Variable(tf.random_normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    'wh1' : tf.Variable(tf.random_normal([input_size_to_hidden, n_hidden])),
    'wo'  : tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([n_conv1])),
    'bc2' : tf.Variable(tf.random_normal([n_conv2])),
    'bh1' : tf.Variable(tf.random_normal([n_hidden])),
    'bo' : tf.Variable(tf.random_normal([n_out]))    
}

In [6]:
# Function to apply Convolution layer
def conv(x, weights, bias, strides=1):
    out = tf.nn.conv2d(x, weights, padding='SAME', strides=[1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

In [7]:
# Function to apply pooling layer
def maxpooling(x, k=2):
    return tf.nn.max_pool(x, padding='SAME',ksize=[1, k, k, 1], strides=[1, k, k, 1])

In [8]:
# Function to implement all layers
def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape = [-1, input_height, input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights['wh1']), biases['bh1'])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob)
    
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output

In [9]:
x = tf.placeholder('float', [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder('float')

pred = cnn(x, weights, biases, keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred, labels = y))

In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
batch_size = 500
for i in range(10):    #lower no of iteration due to kernel restarting in higher no of iterations
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimize], feed_dict= {x:batch_x, y:batch_y, keep_prob : 0.8})
        total_cost += c
    print(i,total_cost)

0 16151.354335784912
1 9381.015480041504
2 5458.960670471191
3 3963.8487091064453
4 2532.4002027511597
5 1870.303109884262
6 1597.2747912704945
7 1200.1950931549072
8 1009.4808544516563
9 729.8013709634542


In [22]:
# Predictions and score on test data
predictions = tf.argmax(pred, 1)
true_label = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, true_label)
predictions_eval, labels, correct_pred = sess.run([predictions, true_label, correct_predictions], feed_dict = {x:mnist.test.images, y:mnist.test.labels,keep_prob : 1.0})
(correct_pred.sum()/len(mnist.test.images))  #Accuracy

0.9788